Import libraries

In [ ]:
import os
import sys
import pandas as pd
import datetime
from tqdm.auto import tqdm
import hyp3_sdk as sdk
from hyp3_sdk import asf_search

Enter Hyp3 credentials

In [ ]:
hyp3 = sdk.HyP3(prompt=True)
my_info = hyp3.my_info()
print(f"Remaining Quota: {my_info['quota']['remaining']}")

Create download list

In [14]:
# Project name
project_name = 'ifg_asc_p81_f1048'

# Set date bounds
start_date = datetime.datetime(2016, 12, 31)
end_date = datetime.datetime(2021, 12, 31)

# Read in 
df = pd.read_csv('..\\slcs_asc_path81_frame1048.csv',usecols=['Granule Name', 'Acquisition Date'])

# Convert and limit by date
df['Acquisition Date'] = pd.to_datetime(df['Acquisition Date'], format='%Y-%m-%dT%H:%M:%S')
df = df.loc[(df['Acquisition Date']>=start_date) & (df['Acquisition Date']<=end_date)]
df = df.sort_values('Acquisition Date')

granules = list(df['Granule Name'])
granules = granules[0:5]
print(f'Will create interferograms for {len(granules)} SLCs')

Will create interferograms for 5 SLCs


Request Ifg generation

In [15]:
insar_jobs = sdk.Batch()
for reference in tqdm(granules):
    neighbors_metadata = asf_search.get_nearest_neighbors(reference, max_neighbors=3)
    for secondary_metadata in neighbors_metadata:
        secondary = secondary_metadata['granuleName']
        insar_jobs += hyp3.submit_insar_job(reference, secondary,
            name=project_name,looks='10x2',include_look_vectors=True,include_inc_map=True,include_dem=True)


100%|██████████| 5/5 [00:32<00:00,  6.46s/it]


Check progress and download

In [ ]:
batch = hyp3.find_jobs(name=project_name)
if batch.complete():
    #filter to only succeeded jobs
    succeeded_jobs = batch.filter_jobs(succeeded=True, running=False, failed=False)
    #download files
    file_list = succeeded_jobs.download_files(location=project_name,create=True)
else:
    #to get updated information
    batch = hyp3.refresh(batch)
    #or to wait until completion and get updated information (which will take a fair bit)
    batch = hyp3.watch(batch)